In [1]:
from ipynb.fs.full.sim import Simulator
from ipynb.fs.full.traffic import Frame
from ipynb.fs.full.logger import Logger
from ipynb.fs.full.topology import Switch

In [2]:
class FRERSplit:
    def __init__(self, split_config: dict):
        """
        split_config:
            stream_id -> list of output ports
        """
        self.split_config = split_config

    def apply(self, switch, frame: Frame):
        ports = self.split_config.get(frame.stream_id)
        if not ports:
            return [(frame, None)]

        replicas = []
        for i, port in enumerate(ports):
            replica = Frame(
                frame_id=frame.frame_id,
                stream_id=frame.stream_id,
                size=frame.size,
                creation_time=frame.creation_time,
                metadata={**frame.metadata, "frer_replica": i}
            )
            replicas.append((replica, port))

        return replicas

In [3]:
class FRERMerge:
    def __init__(self, expire_time: float):
        self.expire_time = expire_time
        self.seen = {}  # stream_id -> {frame_id: timestamp}

    def apply(self, frame: Frame, now: float) -> bool:
        stream_seen = self.seen.setdefault(frame.stream_id, {})

        # Cleanup
        expired = [
            fid for fid, ts in stream_seen.items()
            if now - ts > self.expire_time
        ]
        for fid in expired:
            del stream_seen[fid]

        if frame.frame_id in stream_seen:
            return False

        stream_seen[frame.frame_id] = now
        return True

In [4]:
class FRERSwitch(Switch):
    def __init__(self, name, simulator: Simulator, logger: Logger, routing_table, frer_split: FRERSplit = None, frer_merge: FRERMerge = None):
        super().__init__(name, simulator, logger)
        self.routing_table = routing_table
        self.frer_split = frer_split
        self.frer_merge = frer_merge

    def process(self, frame: Frame, ingress_port: str):
        # MERGE (elimination)
        if self.frer_merge:
            if not self.frer_merge.apply(frame=frame, now=self.sim.time):
                self.logger.log(self.sim.time, self.name, "FRER_DROP", frame, ingress_port)
                return

        # SPLIT (replication)
        if self.frer_split:
            replicas = self.frer_split.apply(self, frame)
        else:
            out_port = self.routing_table.get(frame.stream_id)
            replicas = [(frame, out_port)]

        # Forward replicas
        for replica, port in replicas:
            if port is None:
                port = self.routing_table.get(replica.stream_id)
            if port is None:
                self.logger.log(self.sim.time, self.name, "DROP", replica, None)
            else:
                self.forward(replica, port)